# Practica 3 - Multiclasificadores y selección de variables

## Minería de Datos 2017/2018

* [**Hernán Indíbil de La Cruz Calvo**](https://github.com/Mowstyl)
* [**Alejandro Martín Simón Sánchez**](https://github.com/elssbbboy/)

## Índice
1) [Introducción](#1)

2) [Ensembles](#2)
   
- 2.1 [Implementación básica de un ensemble de manera manual](#21)
   
- 2.2 [Bagging](#22)

- 2.3 [Random Forest](#23)

- 2.4 [Boosting](#24)

- 2.5 [Gradient Boosting Trees](#25)

- 2.6 [Comparación](#26)

3) [Filtrado](#3)

4) [Wrapper](#4)

5) [Algoritmo de selección de parámetros voraz](#5)


## 1. Introducción <a class="anchor" id="1"></a>

Esta práctica se divide en dos partes:

**Multiclasificadores:** estudiaremos la API de scikit para los modelos de tipo ensemble y veremos como entrenar, seleccionar y utilizar estos modelos.

**Selección de variables:** veremos los distintos algoritmos de selección de variables disponibles en scikit y como aplicarlos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
from sklearn.model_selection import cross_val_score

In [4]:
seed=6470
np.random.seed(seed)

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_diabetes
dss = {}
dss['wisconsin'] = load_breast_cancer()
dss['pima'] = load_diabetes()
dir(dss['wisconsin'])

['DESCR', 'data', 'feature_names', 'target', 'target_names']

Como podemos ver, en dss['wisconsin'] tenemos los siguientes atributos:

* DESCR: descripción del dataset, con numero de casos, codificación de los valores perdidos, atributos, información de los mismos además de estadísticos y información adicional sobre la obtención de los datos y su origen.
* data: array de numpy con los valores de cada atributo para cada caso. En prácticas anteriores sería similar al train_atts o test_atts (aún no hemos hecho holdout), cambiando que no se utiliza pandas y no tenemos los nombres de las variables predictoras.
* feature_names: nombres de las variables categóricas, usado como columna en pandas anteriormente. De esta forma, dfs['wisconsin'].data[n][m] indica un valor de la variable dfs['wisconsin'].feature_names[m].
* target: array de numpy con los valores de la clase para cada instancia. No se trabaja con strings, sino que ha sido binarizada para trabajar con 0 y 1 (más eficiente que trabajar con strings).
* target_names: nombres de los distintos valores que toma la variable categórica original. De esta forma si tenemos un 0 en target sabemos que tenemos que se clasifica como target_names[0], en este caso 'malignant'.


In [6]:
dir(dss['pima'])

['DESCR', 'data', 'feature_names', 'target']

Como podemos ver, en dss['pima'] tenemos algunas diferencias con el dataset wisconsin:

* En target ya no tenemos 0 y 1, ya que la variable clase no es categórica, sino una medida cuantitativa de la progresión de la enfermedad un año después de la baseline.
* El atributo target_names no tiene sentido ya que no existe ninguna correspondencia como en wisconsin. La variable clase no es categórica.


In [7]:
from sklearn.model_selection import train_test_split
dssh = {}

for ds in dss:
    strat = None
    if ('target_names' in dir(dss[ds])): # Solo tiene sentido estratificar si la clase es categórica
        strat = dss[ds].target
    
    dssh[ds] = {'train': {}, 'test': {}}
    dssh[ds]['train']['atts'], dssh[ds]['test']['atts'], dssh[ds]['train']['label'], dssh[ds]['test']['label'] = train_test_split(
        dss[ds].data,
        dss[ds].target,
        random_state = seed,
        stratify = strat)

La siguiente función lleva a cabo un muestreo de variables predictoras. Los parámetros que se le pueden pasar son:
* atts: array con los casos, preferiblemente de numpy. train_atts.
* label: array con la variable clase asociada a cada caso. train_label.
* feature_names: nombres de las variables predictoras. Opcional.
* n_samples: número de muestras a generar. Opcional.
* best_features: parámetro que indica si se tomarán las mejores variables para cada muestra. Si hay reemplazo no tiene sentido, ya que las n muestras que genere serán idénticas. Opcional.
* replace: parámetro que indica si el muestreo se realiza con reemplazo. Opcional.
* random_state: parámetro que indica la semilla para generar números aleatorios, solo utilizada con best_features=False. Opcional.
* k: número de variables en cada muestra en caso de haber reemplazo y que best_features=False. Opcional.

Devuelve una tripla con:
* Lista de muestras(solo los casos/atts).
* Lista de muestras(solo la clase/label).
* Lista de listas con nombres de variables predictoras usadas en cada muestra. Si no se especificó feature_names devuelve None.

## 2.  Ensembles <a class="anchor" id="2"></a>

### 2.1  Implementación básica de un ensemble de manera manual <a class="anchor" id="21"></a>

In [8]:
import random
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

def sampleFeatures(atts, label, feature_names=None, n_samples=4, best_features=True, replace=False,
                   random_state=None, k=10):
    if (random_state is not None and not best_features):
        random.seed(random_state)
    kaux = int(atts.shape[1]/n_samples)
    ks = [kaux for i in range(0, n_samples)]
    if (kaux * n_samples < atts.shape[1]):
        ks[-1] += 1
    
    auxatts = atts[:,:]
    fnames = None
    if (feature_names is not None):
        auxfname = feature_names[:]
        fnames = []
    
    sample = []
    labels = []
    
    for s in ks:
        if (best_features):
            sel = SelectKBest(mutual_info_classif, k=s)
            sel.fit(auxatts, label)
            mask = sel.get_support()
        else:
            cols = s
            if (replace):
                cols = k
            samp = random.sample(range(0, auxatts.shape[1]), cols)
            mask = [i in samp for i in range(0, auxatts.shape[1])]
        sample.append(auxatts[:,mask])
        labels.append(label)
        if (feature_names is not None):
            fnames.append(auxfname[mask])
            if(not replace):
                auxfname = auxfname[np.invert(mask)]
        if(not replace):
            auxatts = auxatts[:,np.invert(mask)]
    return np.array(sample), np.array(labels), np.array(fnames)

La siguiente función lleva a cabo el muestreo, sobre variables o sobre casos, con o sin reemplazo. Los parámetros que se le pueden pasar son:
* atts: array con los casos, preferiblemente de numpy. train_atts.
* label: array con la variable clase asociada a cada caso. train_label.
* feature_names: nombres de las variables predictoras. Opcional.
* n_samples: número de muestras a generar. Opcional.
* sample_features: indica si el muestreo se realiza sobre los casos (False) o sobre las variables predictoras (True)
* best_features: parámetro que indica si se tomarán las mejores variables para cada muestra. Si hay reemplazo no tiene sentido, ya que las n muestras que genere serán idénticas. Solo se toma si sample_features=True. Opcional.
* replace: parámetro que indica si el muestreo se realiza con reemplazo (sobre las variables predictoras o sobre los casos según el valor de sample_features). Opcional.
* random_state: parámetro que indica la semilla para generar números aleatorios, solo utilizada con best_features=False. Opcional.
* k: número de variables en cada muestra en caso de haber reemplazo, sample_features=True y best_features=False. Opcional.

Devuelve una tripla con:
* Lista de muestras(solo los casos/atts).
* Lista de muestras(solo la clase/label).
* Lista de listas con nombres de variables predictoras usadas en cada muestra. Si no se especificó feature_names devuelve None.

In [9]:
from sklearn.utils import resample

def ourSample(atts, label, feature_names=None, random_state=None, n_samples=2,
              replace=True, sample_features=False, best_features=True, k=10):
    if (sample_features):
        return sampleFeatures(atts, label, feature_names=feature_names, random_state=random_state,
                              best_features=best_features, n_samples=n_samples, replace=replace, k=k)
    elif(replace):
        sample = []
        labels = []
        fnames = None
        if (feature_names is not None):
            fnames = []
        for i in range(0, n_samples):
            X, y = resample(atts, label, random_state=random_state, n_samples=n_samples, replace=True)
            sample.append(X)
            labels.append(y)
            if (feature_names is not None):
                fnames.append(feature_names)
        return np.array(sample), np.array(labels), np.array(fnames)
    else:
        if (random_state is not None):
            random.seed(random_state)
        ncases = int(atts.shape[0]/n_samples)
        ncss = [ncases for i in range(0, n_samples)]
        if (ncases * n_samples < atts.shape[0]):
            ncss[-1] += 1
        auxatts = atts[:,:]
        auxlabels = label[:]
        sample = []
        labels = []
        fnames = None
        if (feature_names is not None):
            fnames = []
        for nc in ncss:
            samp = random.sample(range(0, auxatts.shape[0]), nc)
            mask = [i in samp for i in range(0, auxatts.shape[0])]
            sample.append(auxatts[mask])
            labels.append(auxlabels[mask])
            if (feature_names is not None):
                fnames.append(feature_names)
            auxatts = auxatts[np.invert(mask)]
            auxlabels = auxlabels[np.invert(mask)]
        return np.array(sample), np.array(labels), np.array(fnames)

Ejemplo de uso:
Vamos a generar una lista de 10 muestras en las que muestreemos las variables predictoras. Cada muestra tendrá 10 variables predictoras escogidas aleatoriamente

In [10]:
a, l, n = ourSample(dssh['wisconsin']['train']['atts'],
                    dssh['wisconsin']['train']['label'],
                    feature_names=dss['wisconsin'].feature_names, n_samples=10,
                    replace=True, best_features=False, sample_features=True)

Así, en a[0] tenemos nuestra primera muestra, donde a[0][0] sería el primer caso

In [11]:
a[0][0]

array([  6.74100000e+01,   2.99500000e-02,   1.20100000e-02,
         6.48100000e-02,   8.61400000e-03,   2.71000000e-02,
         3.45100000e-03,   2.33100000e+01,   7.42200000e+01,
         7.98700000e-02])

en l[0][0] se tendrá el valor de la clase para el primer caso.

In [12]:
l[0][0]

1

En n[0] se tendrá el nombre de las variables predictoras usadas en la primera muestra.

In [13]:
n[0]

array(['mean perimeter', 'mean concavity', 'mean concave points',
       'mean fractal dimension', 'concave points error', 'symmetry error',
       'fractal dimension error', 'worst texture', 'worst perimeter',
       'worst concavity'],
      dtype='<U23')

Una vez listo el método para realizar el muestreo, podemos proceder a diseñar el ensemble.
Tomará como parámetros los necesarios para ajustar el algoritmo del muestreo además de el algoritmo que se usará para el ensemble. Dicho algoritmo vendrá dado con una clase cuyas instancias tengan implementados los métodos set_params, fit y predict. Además, se podrá dar otro parámetro al ensemble llamado params que se podrá usar para indicar los parámetros que se pasan a todas las instancias del algoritmo al crear el ensemble.
Cabe destacar que en lugar de n_samples tendremos n_models, que indicará tanto el número de muestras como el número de estimadores generados.
El ensemble implementará a su vez los métodos fit y predict.

In [14]:
from scipy import stats

class EnsembleHome:
    def __init__(self, est, random_state=None, n_models=10, replace=True,
                 sample_features=False, best_features=True, k=10, params=None):
        self.base = est
        self.rstate = random_state
        self.nmodels = n_models
        self.replace = replace
        self.sfeatures = sample_features
        self.bfeatures = best_features
        self.k = k
        self.params = params
        
        self.ests = []
        for i in range(0, n_models):
            self.ests.append(est())
            if (params is not None):
                self.ests[i].set_params(**params)
        
    def fit(self, atts, label, feature_names=None):
        self.fnames = feature_names
        satts, slabels, sfname = ourSample(atts, label, feature_names=feature_names,
                                           n_samples=self.nmodels, replace=self.replace, random_state=self.rstate,
                                           best_features=self.bfeatures, sample_features=self.sfeatures)
        self.satts = satts
        self.slabels = slabels
        self.sfname = sfname
        for i in range(0, self.nmodels):
            self.ests[i].fit(self.satts[i], self.slabels[i])
    
    def predict(self, atts):
        predictions = []
        for i in range(len(self.ests)):
            mask = [self.fnames[j] in self.sfname[i] for j in range(len(self.fnames))]
            paux = self.ests[i].predict(atts[:,mask])
            predictions.append(paux)
        return stats.mode(predictions)[0][0]
        #return np.stats.mode(predictions)

**Muestreo con reemplazo y sin muestreo de atributos.**

In [15]:
from sklearn import tree
ensemble = EnsembleHome(tree.DecisionTreeClassifier, random_state=seed, n_models=10)

In [16]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [17]:
p = {}
p['Con reemplazo, sin muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

**A continuación se realiza el muestreo sin reemplazo y sin muestreo de atributos.**

In [18]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, random_state=seed, n_models=10)

In [19]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [20]:
p['Sin reemplazo, sin muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

**Lo siguiente es realizar el muestreo sin reemplazo y con muestreo de atributos.**

In [21]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, sample_features=True, best_features=False, random_state=seed, n_models=10)

In [22]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [23]:
p['Sin reemplazo, con muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

**Muestro con reemplazo y con muestreo de atributos**

In [24]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = True, sample_features=True, best_features=False, random_state=seed, n_models=10)

In [25]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [26]:
p['Con reemplazo, con muestreo de atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

**Conjuntos con los mejores atributos**

In [27]:
ensemble = EnsembleHome(tree.DecisionTreeClassifier, replace = False, sample_features=True, best_features=True, random_state=seed, n_models=10)

In [28]:
ensemble.fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], feature_names=dss['wisconsin'].feature_names)

In [29]:
p['Conjuntos con los mejores atributos'] = ensemble.predict(dssh['wisconsin']['test']['atts'])

In [30]:
import sklearn.metrics as metrics
# Metodo que pasado test_label y un diccionario con predicciones indizadas
#   por el algoritmo devuelve una tabla con las metricas utilizadas

def metricTable(test_label, predictions, pos_label):
    algL = []
    accuracyL = []
    recallL = []
    precisionL = []
    f1scoreL = []
    aucL = []

    for alg, prediction in predictions.items():
        algL.append(alg)
        accuracyL.append(metrics.accuracy_score(test_label, prediction))
        recallL.append(metrics.recall_score(test_label, prediction, pos_label=pos_label))
        precisionL.append(metrics.precision_score(test_label, prediction, pos_label=pos_label))
        f1scoreL.append(metrics.f1_score(test_label, prediction, pos_label=pos_label))
        aucL.append(metrics.roc_auc_score(y_true=pd.get_dummies(test_label), y_score=pd.get_dummies(prediction)))

    table = [('Algorithm', algL),
             ('Accuracy', accuracyL),
             ('Recall', recallL),
             ('Precision', precisionL),
             ('F1 Score', f1scoreL),
             ('ROC AUC', aucL)
             ]

    return pd.DataFrame.from_items(table)

def cMatrix(matrix, pos_label, neg_label):
    rowL = [pos_label, neg_label]
    tColL = [matrix[1,1], matrix[0,1]]
    fColL = [matrix[1,0], matrix[0,0]]
    table = [('Actual \ Pred', rowL),
             (pos_label, tColL),
             (neg_label, fColL)]
    return pd.DataFrame.from_items(table)

In [31]:
metricTable(dssh['wisconsin']['test']['label'], p, 0)

,Algorithm,Accuracy,Recall,Precision,F1 Score,ROC AUC
0,"Con reemplazo, sin muestreo de atributos",0.895105,0.830189,0.880000,0.854369,0.881761
1,"Sin reemplazo, sin muestreo de atributos",0.916084,0.849057,0.918367,0.882353,0.902306
2,"Sin reemplazo, con muestreo de atributos",0.937063,0.867925,0.958333,0.910891,0.922851
3,"Con reemplazo, con muestreo de atributos",0.937063,0.886792,0.940000,0.912621,0.926730
4,Conjuntos con los mejores atributos,0.902098,0.849057,0.882353,0.865385,0.891195


___________

### 2.2 Bagging  <a class="anchor" id="22"></a>

La estrategia básica tras el algoritmo de **bagging** es la agregación de distintos clasificadores que han sido aprendidos a partir de muestras obtenidas tras un proceso de bootstraping (muestreo con remplazo). Aquí estudiaremos cómo aprender un modelo de este tipo utilizando scikit.

```
sklearn.ensemble.BaggingClassifier(base_estimator=None, (El clasificador de base que usará el ensemble) 
                                   n_estimators=10, (Numero de modelos que aprenderemos) 
                                   max_samples=1.0, (proporción de la muestra a utilizar)
                                   max_features=1.0, (proporcion de características a  utilizar)
                                   bootstrap=True, (si se utilizará muestreo por remplazo o no)
                                   n_jobs=1, (para utilizar paralelismo, debe soportarlo nuestro entorno)
                                   random_state=None (la semilla)
                                  )
```

In [32]:
from sklearn.ensemble import BaggingClassifier

bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 30, 
                          random_state=seed)

In [33]:
scores_bagg = cross_val_score(bagg, dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_bagg.mean(), scores_bagg.std() * 2))

Accuracy: 0.96 (+/- 0.04)


### 2.3 Random Forest <a class="anchor" id="23"></a>
Una alternativa muy popular al bagging es el clasificador **random forest**. En este caso también se utiliza muestreo con remplazo, pero adicionalmente integra otras técnicas de aleatorización en el aprendizaje de los árboles de decisión para ampliar la generalización del ensemble. Concretamente utiliza una muestra aleatoria de los atributos a la hora de seleccionar cada punto óptimo de corte.

Un clasificador random forest siempre utiliza árboles de decisión como submodelos y por ello no requiere un clasificador base para su definición. Por esa misma razón los hyperparámetros de este clasificador incluyen tanto los correspondientes al ensemble como al árbol de decisión.

```
sklearn.ensemble.RandomForestClassifier(n_estimators=10, (numero de modelos que aprenderemos)
                                        criterion='gini', (parametro del arbol de decision)
                                        max_depth=None, (parametro del arbol de decision)
                                        min_samples_split=2, (parametro del arbol de decision)
                                        min_samples_leaf=1, (parametro del arbol de decision)
                                        min_impurity_split=1e-07, (parametro del arbol de decision)
                                        max_features='auto', (número de atributos que se usan en cada caso:
                                                              puede ser un entero para número exacto, un float
                                                              para proporcion o 'auto', 'sqrt', 'log2' o 'None')
                                        bootstrap=True, (si se utilizará muestreo por remplazo)
                                        n_jobs=1, (para utilizar paralelismo, debe soportarlo nuestro entorno)
                                        random_state=None, (semilla)
                                        )
```

In [34]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, random_state=seed)

In [35]:
scores_rf = cross_val_score(rf, dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std() * 2))

Accuracy: 0.95 (+/- 0.01)


### 2.4 Boosting <a class="anchor" id="24"></a>
La estrategia de boosting es completamente diferente a la anterior, ya que en lugar de reducir la varianza al aprender múltiples clasificadores independientes, realizaremos un proceso iterativo en el que intensificaremos el aprendizaje sobre aquellas instancias más complicadas de aprender para nuestro modelo. 
El algoritmo más conocido se llama `AdaBoost` y está disponible en scikit:

```
sklearn.ensemble.AdaBoostClassifier(base_estimator=None, (El clasificador de base que usará el ensemble) 
                                    n_estimators=50, (numero de modelos que aprenderemos)
                                    learning_rate=1.0, (la contribución del modelo anterior al siguiente)
                                    random_state=None (semilla)
                                    )
```

In [36]:
from sklearn.ensemble import AdaBoostClassifier
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=30)

In [37]:
scores_boost = cross_val_score(boost, dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], cv=3, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_boost.mean(), scores_boost.std() * 2))

Accuracy: 0.92 (+/- 0.00)


### 2.5 Gradient Boosting Trees <a class="anchor" id="25"></a>
Es una generalización del algoritmo de boosting que optimiza funciones de coste en el dominio concreto de los árboles de decisión. Actualmente es una de las técnicas más exitosas para resolver problemas de clasificación complejos *off of the shelf*.

```
sklearn.ensemble.GradientBoostingClassifier(n_estimators=100, (numero de modelos que aprenderemos)
                                            learning_rate=0.1, (la contribución del modelo anterior)
                                            subsample=1.0, (aprender mediante submuestreo)
                                            max_features=None, (submuestrear atributos como en RF)
                                            min_samples_split=2, (parametro del arbol de decision)
                                            min_samples_leaf=1, (parametro del arbol de decision)
                                            max_depth=3, (parametro del arbol de decision)
                                            min_impurity_split=1e-07, (parametro del arbol de decision)
                                            random_state=None, (semilla)
                                            )
```

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=30)

In [39]:
scores_gboost = cross_val_score(gboost, dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'], cv=5, scoring="accuracy")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_gboost.mean(), scores_gboost.std() * 2))

Accuracy: 0.95 (+/- 0.03)


###  2.6 Comparación <a class="anchor" id="26"></a>
Cualquier técnica de ensemble debería ser capaz de mejorar el rendimiento del clasificador base, aunque el rendimiento entre ellas puede depender enormemente del **problema** y de los **parámetros** con los que se hayan configurado las técnicas.

In [40]:
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1]) for x in results])
    display(HTML(report))

In [41]:
results = [("Bagging", scores_bagg.mean()), 
           ("Random Forest", scores_rf.mean()),
           ("Boosting", scores_boost.mean()),
           ("Gradient Boosting", scores_gboost.mean())]
results += [("Casero - " + alg, metrics.accuracy_score(dssh['wisconsin']['test']['label'], p[alg])) for alg in p]
printTable(results)

Bagging,0.957746478873
Random Forest,0.948356807512
Boosting,0.915492957746
Gradient Boosting,0.95072307993
"Casero - Con reemplazo, sin muestreo de atributos",0.895104895105
"Casero - Sin reemplazo, sin muestreo de atributos",0.916083916084
"Casero - Sin reemplazo, con muestreo de atributos",0.937062937063
"Casero - Con reemplazo, con muestreo de atributos",0.937062937063
Casero - Conjuntos con los mejores atributos,0.902097902098


### Estrategias de reducción de varianza vs estrategias de reducción de sesgo (variance and bias)

Aunque la estrategia común detrás de un ensemble sea la de generar un consenso entre varios modelos (mediante agregación o voto por ejemplo), la forma en la que se genera el conjunto de modelos está dirigida por motivaciones muy diferentes para algoritmos como bagging o boosting. Cuando utilizamos una u otra técnica es fundamental conocer el fundamento interno de la técnica de ensembling para seleccionar y aprender correctamente los modelos del ensemble.

En el caso de **bagging** o **random forest** queremos utilizar una función de aprendizaje (C4.5, CART...) y obtener modelos "diversos" a partir de los mismos datos para reducir el error obtenido mediante **varianza**. Formalmente hablamos de clasificadores que individualmente tengan poder de generalización pero que al mismo tiempo estén lo menos correlados entre ellos. Es por ello que se utilizan diversas técnicas de submuestreo y aleatorización de los modelos. 

En el caso de los algoritmos de **boosting** la estrategia es opuesta, dado un problema complejo queremos reducir el sesgo en nuestros modelos. Por ello en lugar de aleatorización guíamos la función de aprendizaje para que incida en aquellos ejemplos que sean más dificiles de generalizar. La técnica general es utilizar sobremuestreo o pesos para que sobreajustar el clasificador en aquellos casos conflictivos, de manera más formal o sofisticada se puede establecer un problema de optimización de funciones de coste.

___
Es por ello que para el **bagging** es interesante utilizar 'strong learners' clasificadores con un gran poder predictivo en sus parámetros que den lugar a **modelos con mucha varianza, ya que la varianza se reduce.**

Para el caso de algoritmos de **boosting** es interesante utilizar clasificadores 'weak learners' con un poder de generalizacion y parametros no muy complejos ni sobreajustados. Así se le ofrece al algoritmo de **boosting** espacio para optimizar dichos parametros y teóricamente transformar al clasificador base en un strong learner. Por ejemplo, arboles de decision muy poco profundos (shallow trees).



## 3. Filtrado <a class="anchor" id="3"></a>

In [42]:
fss = SelectKBest(mutual_info_classif, k=int(round(dssh['wisconsin']['train']['atts'].shape[1]/2))).fit(dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'])

# We can obtain the mask of the selected attributes
print(dss['wisconsin'].feature_names[fss.get_support()])

['mean radius' 'mean perimeter' 'mean area' 'mean compactness'
 'mean concavity' 'mean concave points' 'radius error' 'perimeter error'
 'area error' 'worst radius' 'worst perimeter' 'worst area'
 'worst compactness' 'worst concavity' 'worst concave points']


In [43]:
def filterAtts(atts, label, feature_names = None, cls = mutual_info_classif, k = 10):
    if (not ('chi2' in str(cls)) or not (atts < 0).any()):
        fss = SelectKBest(cls, k=k).fit(atts, label)
        # We can obtain the mask of the selected attributes
        fn = feature_names
        if (str(type(feature_names)) == "<class 'list'>"):
            fn = np.array(feature_names)
        print(fn[fss.get_support()])
        # We can transform the dataset to project only the selected attributes
        print(fss.transform(atts).shape[1])
        return fss
    else:
        print ("chi2 doesn't work with negative values!")

In [44]:
filteredAtts = {}
for i in dssh:
    print(i + ":")
    numVar = int(round(dssh[i]['train']['atts'].shape[1]/2))
    filteredAtts[i] = filterAtts(dssh[i]['train']['atts'],
                                 dssh[i]['train']['label'],
                                 feature_names=dss[i].feature_names,
                                 k = numVar)
    print()

wisconsin:
['mean radius' 'mean perimeter' 'mean area' 'mean compactness'
 'mean concavity' 'mean concave points' 'radius error' 'perimeter error'
 'area error' 'worst radius' 'worst perimeter' 'worst area'
 'worst compactness' 'worst concavity' 'worst concave points']
15

pima:
['sex' 'bmi' 's3' 's4' 's5']
5



In [45]:
from sklearn.feature_selection import chi2

for i in dssh:
    print(i + ":")
    numVar = int(round(dssh[i]['train']['atts'].shape[1]/2))
    filteredAtts[i] = filterAtts(dssh[i]['train']['atts'],
                                 dssh[i]['train']['label'],
                                 feature_names=dss[i].feature_names,
                                 k=numVar,
                                 cls=chi2)
    print()

wisconsin:
['mean radius' 'mean texture' 'mean perimeter' 'mean area' 'mean concavity'
 'radius error' 'perimeter error' 'area error' 'worst radius'
 'worst texture' 'worst perimeter' 'worst area' 'worst compactness'
 'worst concavity' 'worst concave points']
15

pima:
chi2 doesn't work with negative values!



## 4. Wrapper <a class="anchor" id="4"></a>

In [46]:
def ourWrapperAux(cls, atts, label, mask, checked):
    maskL = []
    for i in range(len(mask)):
        if(mask[i]):
            maskaux = mask[:]
            maskaux[i] = False
            if (maskL not in checked):
                maskL.append(maskaux)
                checked.append(maskL)
    
    scores = []
    scores.append((cross_val_score(cls, atts[:,mask], label).mean(), mask))
    for m in maskL:
        if (any(m)):
            scores.append(ourWrapperAux(cls, atts, label, m, checked))
    return max(scores)

In [47]:
def ourWrapper(cls, atts, label):
    mask = [True for i in range(0,atts.shape[1])]
    check = []
    res = ourWrapperAux(cls, atts, label, mask, check)
    return res

In [48]:
clas = tree.DecisionTreeClassifier(random_state = seed)
result = ourWrapper(clas, dssh['wisconsin']['train']['atts'], dssh['wisconsin']['train']['label'])
print(dss['wisconsin'].feature_names[result[1]])
print('Score: ' + str(result[0]))

['worst area' 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Score: 0.934272300469


## 5. Algoritmo de selección de parámetros voraz <a class="anchor" id="5"></a>

Para aplicar un esquema voraz debemos tener en cuenta tres funciones que hay que especificar:

* seleccionar: Permite determinar que elemento de C seleccionar en cada paso del algoritmo.
* es prometedora: Determina si una solucion x puede extenderse para formar una solucion factible.
* es factible: Determina si una solucion x es factible, es decir, si se ajusta a las restricciones del problema.

Para poder realizarlas lo primero que se debe hacer es determinar como se va a codificar la solución, en este caso utilizamos una máscara que indica si un atributo está incluido o no en la solución.

Lo primero que se realiza es un ranking de las variables en función de la información mutua, y posteriormente, comenzando por las de mayor peso son evaluadas. Si mejora se añade a la solución, en caso contrario esa variable y todas las que estén por debajo quedan descartadas y termina la ejecución del algoritmo.

In [49]:
from sklearn.feature_selection import mutual_info_classif

# We can compute the scores and ranking directly:

def ranking(atts, label, feature_names = None, cls = mutual_info_classif, 
            random_state = None):
    if ('mutual_info_classif' in str(cls)):
        scores = list(cls(atts, label, random_state=random_state))
    else:
        scores = list(cls(atts, label))
    if (len(scores) == 2 and atts.shape[1] != 2):
        scores = scores[0] # En sklearn hay varias clasificaciones que devuelven
        # no solo los scores, sino un array de p-valores.
    names = list(feature_names)
    ranks = sorted( list(zip(scores, names)), reverse=True )
    return ranks

In [50]:
# Prueba ranking
ranking(dssh['wisconsin']['train']['atts'],
        dssh['wisconsin']['train']['label'],
       feature_names = dss['wisconsin'].feature_names,
       cls=mutual_info_classif)

[(0.49086511932726973, 'worst perimeter'),
 (0.46735131033834398, 'worst radius'),
 (0.46129259897582275, 'worst area'),
 (0.45061762610495726, 'worst concave points'),
 (0.43925445869247559, 'mean concave points'),
 (0.42046993708407365, 'mean perimeter'),
 (0.3817026110091668, 'mean radius'),
 (0.38042074415854388, 'mean concavity'),
 (0.37953898514885154, 'mean area'),
 (0.35996521750694987, 'area error'),
 (0.33984926646691815, 'worst concavity'),
 (0.27155716527429252, 'perimeter error'),
 (0.25919475821668869, 'radius error'),
 (0.2383517022850099, 'worst compactness'),
 (0.21122035875400647, 'mean compactness'),
 (0.16887223636501547, 'worst texture'),
 (0.15974418900132203, 'concave points error'),
 (0.1363522310238785, 'mean texture'),
 (0.12480012241200922, 'concavity error'),
 (0.092197899731085275, 'worst symmetry'),
 (0.091346535996676748, 'mean smoothness'),
 (0.079550583497442373, 'worst smoothness'),
 (0.070737887481229444, 'worst fractal dimension'),
 (0.07043278297320

In [51]:
def greedySelect(atts, label, feature_names = None, cls = None,
                 random_state = None, scr = mutual_info_classif):
    rank = ranking(atts, label, feature_names = feature_names, cls = scr, random_state = random_state)
    mask = [False for i in range(atts.shape[1])]
    score = 0
    fnames = np.array(feature_names)
    for i in range(atts.shape[1]):
        nm = mask[:]
        fname = rank[i][1]
        index = np.where(fnames==fname)[0][0]
        nm[index] = True
        ns = cross_val_score(cls, atts[:,nm], label).mean()
        if (ns > score):
            score = ns
            mask = nm
    return mask

In [52]:
m = {}
for i in dssh:
    m[i] = greedySelect(dssh[i]['train']['atts'],
                     dssh[i]['train']['label'],
                     feature_names = dss[i].feature_names,
                     scr=mutual_info_classif, 
                     cls=tree.DecisionTreeClassifier(random_state=seed),
                     random_state = seed)

In [53]:
dss['wisconsin'].feature_names[m['wisconsin']]

array(['worst perimeter', 'worst area', 'worst concave points'],
      dtype='<U23')

In [54]:
np.array(dss['pima'].feature_names)[m['pima']]

array(['sex'],
      dtype='<U3')